### Required Libraries

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

import h5py

from sklearn.preprocessing import scale

from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, Reshape, Flatten, MaxPooling2D, Dropout
from keras.utils import np_utils

Using TensorFlow backend.


### Read and filter data

In [2]:
df = pd.read_csv('fer2013.csv')

labels = df[df.columns[0]]
#changing output labels to categorical format
labels = np_utils.to_categorical(labels)
image_pixels_string = df[df.columns[1]]
image = np.zeros((df.shape[0], 48*48))

#forming image matrix of size 1, 48*48
for ix in range(image.shape[0]):
    temp = image_pixels_string[ix].split(' ')
    for iy in range(image.shape[1]):
        image[ix, iy] = int(temp[iy])

#usig sklearn to normalize each feature
image = scale(image, axis=0)

#forming training dataset and reshaping it to format accepted by input layer
X_train_num = (df[df['Usage']=='Training']).count()[0]
X_train = image[:X_train_num, :].reshape((X_train_num, 48, 48, 1))
y_train = labels[:X_train_num, :]

#forming tcross validation dataset and reshaping it to format accepted by input layer
X_cv_num = (df[df['Usage']=='PublicTest']).count()[0]
X_cv = image[:X_cv_num, :].reshape((X_cv_num, 48, 48, 1))
y_cv = labels[:X_cv_num, :]

#forming ttesting dataset and reshaping it to format accepted by input layer
X_test_num = (df[df['Usage']=='PrivateTest']).count()[0]
X_test = image[:X_test_num, :].reshape((X_test_num, 48, 48, 1))
y_test = labels[:X_test_num, :]

### Model Creation

In [3]:
model = Sequential()

model.add(Conv2D(64, (3, 3), input_shape=(48, 48, 1), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dense(7, activation='softmax'))

model.summary()
#keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='categorical_crossentropy', optimizer='adam',
                  metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 46, 46, 64)        640       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 44, 44, 64)        36928     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 42, 42, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 21, 21, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 19, 19, 32)        18464     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 17, 17, 32)        9248      
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 15, 15, 32)        9248      
__________

### Model fit and evaluation

In [4]:
hist = model.fit(X_train, y_train,
                     epochs=5,
                     shuffle=True,
                      batch_size=256,
                     validation_data=(X_cv, y_cv))

print('***********************************************************************')
score = model.evaluate(X_test, y_test, verbose=0)
print('Testing loss : {}'.format(score[0]))
print('Testing accuracy : {}'.format(score[1]))
print('***********************************************************************')

Train on 28709 samples, validate on 3589 samples
Epoch 1/5
28709/28709 [==============================] - 500s 17ms/step - loss: 1.7442 - acc: 0.2891 - val_loss: 1.5752 - val_acc: 0.3828
Epoch 2/5
28709/28709 [==============================] - 519s 18ms/step - loss: 1.5166 - acc: 0.4090 - val_loss: 1.3753 - val_acc: 0.4712
Epoch 3/5
28709/28709 [==============================] - 534s 19ms/step - loss: 1.3733 - acc: 0.4725 - val_loss: 1.2534 - val_acc: 0.5208
Epoch 4/5
28709/28709 [==============================] - 507s 18ms/step - loss: 1.2879 - acc: 0.5088 - val_loss: 1.2001 - val_acc: 0.5394
Epoch 5/5
28709/28709 [==============================] - 505s 18ms/step - loss: 1.2179 - acc: 0.5330 - val_loss: 1.1189 - val_acc: 0.5759


In [14]:
print('**************************************')
#score = model.evaluate(X_test, y_test, verbose=0)
print('Testing loss : {}    *'.format(score[0]))
print('Testing accuracy : {}'.format(score[1]))
print('**************************************')

**************************************
Testing loss : 1.118852109555648    *
Testing accuracy : 0.5759264419141271
**************************************


### Saving model for prediction

In [5]:
model.save('live_emotion_recognition_model.h5')  # creates a HDF5 file 'my_model.h5'
fhandle = h5py.File('live_emotion_recognition_model.h5', 'r+')
del fhandle['optimizer_weights']
fhandle.close()